In [1]:
import six
from keras.models import Model
from keras.layers import (
    Input,
    Activation,
    Dense,
    Flatten
)

from keras.layers.convolutional import (
    Conv2D,
    MaxPooling2D,
    AveragePooling2D
)
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

Using TensorFlow backend.


In [2]:
import os
import csv
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle
%matplotlib inline

In [3]:
import imageio

In [4]:
def _bn_relu(input):
    """Helper to build a BN -> relu block
    """
    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
    return Activation("relu")(norm)


def _conv_bn_relu(**conv_params):
    """Helper to build a conv -> BN -> relu block
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)

    return f


def _bn_relu_conv(**conv_params):
    """Helper to build a BN -> relu -> conv block.
    This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(activation)

    return f


def _shortcut(input, residual):
    """Adds a shortcut between input and residual block and merges them with "sum"
    """
    # Expand channels of shortcut to match residual.
    # Stride appropriately to match residual (width, height)
    # Should be int if network architecture is correctly configured.
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

    shortcut = input
    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],
                          kernel_size=(1, 1),
                          strides=(stride_width, stride_height),
                          padding="valid",
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(input)

    return add([shortcut, residual])


def _residual_block(block_function, filters, repetitions, is_first_layer=False):
    """Builds a residual block with repeating bottleneck blocks.
    """
    def f(input):
        for i in range(repetitions):
            init_strides = (1, 1)
            if i == 0 and not is_first_layer:
                init_strides = (2, 2)
            input = block_function(filters=filters, init_strides=init_strides,
                                   is_first_block_of_first_layer=(is_first_layer and i == 0))(input)
        return input

    return f


def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv1 = Conv2D(filters=filters, kernel_size=(3, 3),
                           strides=init_strides,
                           padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=l2(1e-4))(input)
        else:
            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3),
                                  strides=init_strides)(input)

        residual = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
        return _shortcut(input, residual)

    return f


def bottleneck(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Bottleneck architecture for > 34 layer resnet.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf

    Returns:
        A final conv layer of filters * 4
    """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv_1_1 = Conv2D(filters=filters, kernel_size=(1, 1),
                              strides=init_strides,
                              padding="same",
                              kernel_initializer="he_normal",
                              kernel_regularizer=l2(1e-4))(input)
        else:
            conv_1_1 = _bn_relu_conv(filters=filters, kernel_size=(1, 1),
                                     strides=init_strides)(input)

        conv_3_3 = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv_1_1)
        residual = _bn_relu_conv(filters=filters * 4, kernel_size=(1, 1))(conv_3_3)
        return _shortcut(input, residual)

    return f


def _handle_dim_ordering():
    global ROW_AXIS
    global COL_AXIS
    global CHANNEL_AXIS
    if K.image_dim_ordering() == 'tf':
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3
    else:
        CHANNEL_AXIS = 1
        ROW_AXIS = 2
        COL_AXIS = 3


def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifier))
        return res
    return identifier


class ResnetBuilder(object):
    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions):
        """Builds a custom ResNet like architecture.

        Args:
            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)
            num_outputs: The number of outputs at final softmax layer
            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.
                The original paper used basic_block for layers < 50
            repetitions: Number of repetitions of various block units.
                At each block unit, the number of filters are doubled and the input size is halved

        Returns:
            The keras `Model`.
        """
        _handle_dim_ordering()
        if len(input_shape) != 3:
            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")

        # Permute dimension order if necessary
        if K.image_dim_ordering() == 'tf':
            input_shape = (input_shape[1], input_shape[2], input_shape[0])

        # Load function from str if needed.
        block_fn = _get_block(block_fn)

        input = Input(shape=input_shape)
        conv1 = _conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(input)
        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)

        block = pool1
        filters = 64
        for i, r in enumerate(repetitions):
            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)
            filters *= 2

        # Last activation
        block = _bn_relu(block)

        # Classifier block
        block_shape = K.int_shape(block)
        pool2 = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]),
                                 strides=(1, 1))(block)
        flatten1 = Flatten()(pool2)
        dense = Dense(units=num_outputs, kernel_initializer="he_normal",
                      activation="softmax")(flatten1)

        model = Model(inputs=input, outputs=dense)
        return model

    @staticmethod
    def build_resnet_18(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [2, 2, 2, 2])

    @staticmethod
    def build_resnet_34(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_50(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_101(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 23, 3])


In [5]:
with open('../cl3403/pickle/y_train1.pickle', mode = 'rb') as f:
    y_train1 = pickle.load(f)
with open('../cl3403/pickle/y_train2.pickle', mode = 'rb') as f:
    y_train2 = pickle.load(f)
with open('../cl3403/pickle/y_train3.pickle', mode = 'rb') as f:
    y_train3 = pickle.load(f)
with open('../cl3403/pickle/y_train4.pickle', mode = 'rb') as f:
    y_train4 = pickle.load(f)
with open('../cl3403/pickle/y_train5.pickle', mode = 'rb') as f:
    y_train5 = pickle.load(f)
with open('../cl3403/pickle/y_train6.pickle', mode = 'rb') as f:
    y_train6 = pickle.load(f)
with open('../cl3403/pickle/y_train7.pickle', mode = 'rb') as f:
    y_train7 = pickle.load(f)
with open('../cl3403/pickle/y_train8.pickle', mode = 'rb') as f:
    y_train8 = pickle.load(f)
with open('../cl3403/pickle/y_train9.pickle', mode = 'rb') as f:
    y_train9 = pickle.load(f)
with open('../cl3403/pickle/y_train10.pickle', mode = 'rb') as f:
    y_train10 = pickle.load(f)

In [6]:
with open('../cl3403/pickle/X_train1.pickle', mode = 'rb') as f:
    X_train1 = pickle.load(f)
with open('../cl3403/pickle/X_train2.pickle', mode = 'rb') as f:
    X_train2 = pickle.load(f)

In [7]:
with open('../cl3403/pickle/X_train3.pickle', mode = 'rb') as f:
    X_train3 = pickle.load(f)
with open('../cl3403/pickle/X_train4.pickle', mode = 'rb') as f:
    X_train4 = pickle.load(f)
with open('../cl3403/pickle/X_train5.pickle', mode = 'rb') as f:
    X_train5 = pickle.load(f)

In [8]:
X_train = np.vstack((X_train1, X_train2))
X_train = np.vstack((X_train, X_train3))
X_train = np.vstack((X_train, X_train4))
X_train = np.vstack((X_train, X_train5))

In [9]:
y_train = np.hstack((y_train1, y_train2))
y_train = np.hstack((y_train, y_train3))
y_train = np.hstack((y_train, y_train4))
y_train = np.hstack((y_train, y_train5))

In [10]:
train1 = []
train2 = []
for i in range(len(X_train)):
    if (i % 4) != 0:
        train1.append(i)
    else:
        train2.append(i)

In [12]:
X_train_true1 = X_train[train1]

In [13]:
y_train_true1 = y_train[train1]

In [17]:
X_train_true1

array([[[[0.34117647, 0.31372549, 0.2       ],
         [0.38431373, 0.35686275, 0.24313725],
         [0.35294118, 0.32941176, 0.20392157],
         ...,
         [0.34509804, 0.3372549 , 0.18823529],
         [0.39215686, 0.38431373, 0.23529412],
         [0.36078431, 0.35294118, 0.20392157]],

        [[0.37647059, 0.34901961, 0.23529412],
         [0.2627451 , 0.23529412, 0.12156863],
         [0.25882353, 0.23529412, 0.10196078],
         ...,
         [0.35686275, 0.34509804, 0.20784314],
         [0.40784314, 0.4       , 0.25098039],
         [0.34901961, 0.34117647, 0.19215686]],

        [[0.30196078, 0.28235294, 0.16470588],
         [0.28627451, 0.26666667, 0.14117647],
         [0.32156863, 0.30588235, 0.16078431],
         ...,
         [0.37254902, 0.36078431, 0.22352941],
         [0.41568627, 0.40784314, 0.25882353],
         [0.34509804, 0.3372549 , 0.18823529]],

        ...,

        [[0.51764706, 0.39607843, 0.28627451],
         [0.51764706, 0.39607843, 0.28627451]

In [18]:
del X_train, y_train, X_train1, X_train2, X_train3, X_train4, X_train5

In [19]:
with open('../cl3403/pickle/X_train6.pickle', mode = 'rb') as f:
    X_train6 = pickle.load(f)
with open('../cl3403/pickle/X_train7.pickle', mode = 'rb') as f:
    X_train7 = pickle.load(f)

In [20]:
with open('../cl3403/pickle/X_train8.pickle', mode = 'rb') as f:
    X_train8 = pickle.load(f)
with open('../cl3403/pickle/X_train9.pickle', mode = 'rb') as f:
    X_train9 = pickle.load(f)
with open('../cl3403/pickle/X_train10.pickle', mode = 'rb') as f:
    X_train10 = pickle.load(f)

In [21]:
X_train = np.vstack((X_train6, X_train7))
X_train = np.vstack((X_train, X_train8))
X_train = np.vstack((X_train, X_train9))
X_train = np.vstack((X_train, X_train10))

In [22]:
y_train = np.hstack((y_train6, y_train7))
y_train = np.hstack((y_train, y_train8))
y_train = np.hstack((y_train, y_train9))
y_train = np.hstack((y_train, y_train10))

In [23]:
train3 = []
train4 = []
for i in range(len(X_train)):
    if (i % 4) != 0:
        train3.append(i)
    else:
        train4.append(i)

In [25]:
del X_train6, X_train7, X_train8, X_train9, X_train10

In [26]:
X_train_true2 = X_train[train3]

In [31]:
y_train_true2 = y_train[train3]

In [35]:
del X_train

In [36]:
X_train_true = np.vstack((X_train_true1, X_train_true2))
y_train_true = np.hstack((y_train_true1, y_train_true2))

In [39]:
model = ResnetBuilder.build_resnet_34((3, 64, 64), 14951)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
hist = model.fit(X_train_true, y_train_true, batch_size = 2000, epochs = 10) 

Epoch 1/10
698826/698826 [==============================] - 366s 523us/step - loss: 6.0208 - acc: 0.2436
Epoch 2/10
698826/698826 [==============================] - 360s 515us/step - loss: 3.3650 - acc: 0.5019
Epoch 3/10
698826/698826 [==============================] - 358s 512us/step - loss: 2.3915 - acc: 0.6389
Epoch 4/10
698826/698826 [==============================] - 357s 511us/step - loss: 1.8984 - acc: 0.7183
Epoch 5/10
698826/698826 [==============================] - 359s 514us/step - loss: 1.6109 - acc: 0.7697
Epoch 6/10
698826/698826 [==============================] - 358s 512us/step - loss: 1.4207 - acc: 0.8051
Epoch 7/10
698826/698826 [==============================] - 358s 512us/step - loss: 1.2825 - acc: 0.8315
Epoch 8/10
698826/698826 [==============================] - 357s 512us/step - loss: 1.1804 - acc: 0.8510
Epoch 9/10
698826/698826 [==============================] - 360s 515us/step - loss: 1.1035 - acc: 0.8670
Epoch 10/10
698826/698826 [============================

In [41]:
hist = model.fit(X_train_true, y_train_true, batch_size = 2000, epochs = 3) 

Epoch 1/3
698826/698826 [==============================] - 357s 511us/step - loss: 0.9948 - acc: 0.8890
Epoch 2/3
698826/698826 [==============================] - 358s 512us/step - loss: 0.9572 - acc: 0.8964
Epoch 3/3
698826/698826 [==============================] - 358s 512us/step - loss: 0.9239 - acc: 0.9034


In [43]:
hist = model.fit(X_train_true, y_train_true, batch_size = 2000, epochs = 3) 

Epoch 1/3
698826/698826 [==============================] - 357s 511us/step - loss: 0.8911 - acc: 0.9098
Epoch 2/3
698826/698826 [==============================] - 357s 510us/step - loss: 0.8696 - acc: 0.9137
Epoch 3/3
698826/698826 [==============================] - 357s 511us/step - loss: 0.8530 - acc: 0.9165


In [44]:
hist = model.fit(X_train_true, y_train_true, batch_size = 2000, epochs = 2) 

Epoch 1/2
698826/698826 [==============================] - 357s 510us/step - loss: 0.8333 - acc: 0.9201
Epoch 2/2
698826/698826 [==============================] - 358s 513us/step - loss: 0.8172 - acc: 0.9234


In [45]:
y_train_true

array([    0,     0,     0, ..., 14950, 14950, 14950])

In [46]:
del X_train_true, y_train_true

In [47]:
with open('../cl3403/pickle/X_truetest.pickle', mode = 'rb') as f:
    X_truetest = pickle.load(f)
with open('../cl3403/pickle/y_truetest.pickle', mode = 'rb') as f:
    y_truetest = pickle.load(f)

In [48]:
pred = model.predict(X_truetest)

In [49]:
landmark_id_pred = pred.argmax(axis = 1)
confidence = pred.max(axis = 1)
df = pd.DataFrame({'id': np.array(y_truetest),
                   'landmark_id': landmark_id_pred,
                   'confidence': confidence})

In [50]:
df['landmarks'] = df.apply(lambda x: str(x['landmark_id']) + ' ' + str(x['confidence']), axis = 1)

In [51]:
df

,confidence,id,landmark_id,landmarks
0,0.260084,000088da12d664db,14255,14255 0.26008448004722595
1,0.445931,0001623c6d808702,6395,6395 0.4459306299686432
2,0.708954,0001bbb682d45002,955,955 0.7089539170265198
3,0.174994,0002362830cfe3a3,7420,7420 0.17499412596225739
4,0.430499,000270c9100de789,4404,4404 0.4304988384246826
5,0.546776,0002b0fab5d3ccc4,11491,11491 0.546776294708252
6,0.426879,000396be3c24830a,10184,10184 0.42687922716140747
7,0.294913,000506dc6ab3a40e,6125,6125 0.2949131727218628
8,0.927360,0005292fc4b005a3,1644,1644 0.9273603558540344
9,0.301322,0005456a82264bc8,4404,4404 0.30132150650024414


In [52]:
df = df.drop(['confidence', 'landmark_id'], axis = 1)

In [53]:
df

,id,landmarks
0,000088da12d664db,14255 0.26008448004722595
1,0001623c6d808702,6395 0.4459306299686432
2,0001bbb682d45002,955 0.7089539170265198
3,0002362830cfe3a3,7420 0.17499412596225739
4,000270c9100de789,4404 0.4304988384246826
5,0002b0fab5d3ccc4,11491 0.546776294708252
6,000396be3c24830a,10184 0.42687922716140747
7,000506dc6ab3a40e,6125 0.2949131727218628
8,0005292fc4b005a3,1644 0.9273603558540344
9,0005456a82264bc8,4404 0.30132150650024414


In [54]:
df_sample = pd.read_csv('sample_submission.csv')[['id']]
df_final = df_sample.merge(df, how = 'left', on = 'id')
df_final.to_csv('submission20.csv', index = False)

In [55]:
from keras.models import load_model

In [67]:
import h5py

In [69]:
model.save('my_model.h5')

ImportError: `save_model` requires h5py.

In [70]:
import pickle

In [71]:
with open('mymodel.pickle', mode='wb') as f:
    pickle.dump(model, f,protocol=4)

PicklingError: Can't pickle <class 'module'>: attribute lookup module on builtins failed

In [74]:
model.save_weights('model_weights.h5')

ImportError: `save_weights` requires h5py.

In [ ]:
X_train1 = np.vstack((X_train1, X_test))

In [11]:
y_train1 = np.hstack((y_train1, y_test))
y_train1 = np.array([y_train1]).T

In [14]:
X_train2 = np.vstack((X_train2, X_test))
y_train2 = np.hstack((y_train2, y_test))
y_train2 = np.array([y_train2]).T

In [15]:
X_train3 = np.vstack((X_train3, X_test))
y_train3 = np.hstack((y_train3, y_test))
y_train3 = np.array([y_train3]).T

In [16]:
X_train4 = np.vstack((X_train4, X_test))
y_train4 = np.hstack((y_train4, y_test))
y_train4 = np.array([y_train4]).T

In [17]:
X_train5 = np.vstack((X_train5, X_test))
y_train5 = np.hstack((y_train5, y_test))
y_train5 = np.array([y_train5]).T

In [18]:
for i in range(10):
    hist = model.fit(X_train1, y_train1, batch_size = 2000, epochs = 1) 
    hist = model.fit(X_train2, y_train2, batch_size = 2000, epochs = 1) 
    hist = model.fit(X_train3, y_train3, batch_size = 2000, epochs = 1) 
    hist = model.fit(X_train4, y_train4, batch_size = 2000, epochs = 1) 
    hist = model.fit(X_train5, y_train5, batch_size = 2000, epochs = 1)     

Epoch 1/1
210934/210934 [==============================] - 88s 419us/step - loss: 7.3083 - acc: 0.1368
Epoch 1/1
210959/210959 [==============================] - 85s 402us/step - loss: 5.3143 - acc: 0.3025
Epoch 1/1
210935/210935 [==============================] - 85s 403us/step - loss: 4.4771 - acc: 0.3659
Epoch 1/1
210969/210969 [==============================] - 85s 403us/step - loss: 3.1671 - acc: 0.5404
Epoch 1/1
210953/210953 [==============================] - 85s 401us/step - loss: 2.9289 - acc: 0.5439
Epoch 1/1
210934/210934 [==============================] - 84s 397us/step - loss: 2.7081 - acc: 0.5640
Epoch 1/1
210959/210959 [==============================] - 84s 399us/step - loss: 2.1329 - acc: 0.6650
Epoch 1/1
210935/210935 [==============================] - 84s 399us/step - loss: 1.9407 - acc: 0.7043
Epoch 1/1
210969/210969 [==============================] - 84s 400us/step - loss: 1.4372 - acc: 0.8056
Epoch 1/1
210953/210953 [==============================] - 84s 400us/step

KeyboardInterrupt: 

In [5]:
with open('../cl3403/pickle/X_test.pickle', mode = 'rb') as f:
    X_test = pickle.load(f)
with open('../cl3403/pickle/y_test.pickle', mode = 'rb') as f:
    y_test = pickle.load(f)

In [21]:
with open('../cl3403/pickle/X_train6.pickle', mode = 'rb') as f:
    X_train6 = pickle.load(f)
with open('../cl3403/pickle/X_train7.pickle', mode = 'rb') as f:
    X_train7 = pickle.load(f)

In [22]:
with open('../cl3403/pickle/X_train8.pickle', mode = 'rb') as f:
    X_train8 = pickle.load(f)
with open('../cl3403/pickle/X_train9.pickle', mode = 'rb') as f:
    X_train9 = pickle.load(f)

In [23]:
with open('../cl3403/pickle/X_train10.pickle', mode = 'rb') as f:
    X_train10 = pickle.load(f)

In [24]:
X_train6 = np.vstack((X_train6, X_test))
y_train6 = np.hstack((y_train6, y_test))
y_train6 = np.array([y_train6]).T

In [25]:
X_train7 = np.vstack((X_train7, X_test))
y_train7 = np.hstack((y_train7, y_test))
y_train7 = np.array([y_train7]).T

In [26]:
X_train8 = np.vstack((X_train8, X_test))
y_train8 = np.hstack((y_train8, y_test))
y_train8 = np.array([y_train8]).T

In [27]:
X_train9 = np.vstack((X_train9, X_test))
y_train9 = np.hstack((y_train9, y_test))
y_train9 = np.array([y_train9]).T

In [28]:
X_train10 = np.vstack((X_train10, X_test))
y_train10 = np.hstack((y_train10, y_test))
y_train10 = np.array([y_train10]).T

In [29]:
y_train10.shape

(147832, 1)

In [30]:
for i in range(3):
    hist = model.fit(X_train6, y_train6, batch_size = 2000, epochs = 1) 
    hist = model.fit(X_train7, y_train7, batch_size = 2000, epochs = 1) 
    hist = model.fit(X_train8, y_train8, batch_size = 2000, epochs = 1) 
    hist = model.fit(X_train9, y_train9, batch_size = 2000, epochs = 1) 
    hist = model.fit(X_train10, y_train10, batch_size = 2000, epochs = 1) 

Epoch 1/1
210917/210917 [==============================] - 84s 398us/step - loss: 1.6922 - acc: 0.7790
Epoch 1/1
210891/210891 [==============================] - 84s 399us/step - loss: 1.1018 - acc: 0.8840
Epoch 1/1
210929/210929 [==============================] - 85s 403us/step - loss: 1.5873 - acc: 0.8018
Epoch 1/1
210950/210950 [==============================] - 80s 378us/step - loss: 1.4797 - acc: 0.8215
Epoch 1/1
147832/147832 [==============================] - 58s 394us/step - loss: 1.0942 - acc: 0.9008
Epoch 1/1
210917/210917 [==============================] - 85s 402us/step - loss: 1.0632 - acc: 0.8890
Epoch 1/1
210891/210891 [==============================] - 85s 402us/step - loss: 0.8120 - acc: 0.9331
Epoch 1/1
210929/210929 [==============================] - 85s 401us/step - loss: 1.0982 - acc: 0.8736
Epoch 1/1
210950/210950 [==============================] - 78s 371us/step - loss: 1.0632 - acc: 0.8818
Epoch 1/1
147832/147832 [==============================] - 55s 373us/step

In [31]:
del X_train6, X_train7, X_train8, X_train9, X_train10, y_train6, y_train7, y_train8, y_train9, y_train10

In [39]:
df

,confidence,id,landmark_id,landmarks
0,0.224375,000088da12d664db,14591,14591 0.2243746519088745
1,0.687623,0001623c6d808702,14187,14187 0.6876228451728821
2,0.236064,0001bbb682d45002,14442,14442 0.23606419563293457
3,0.092695,0002362830cfe3a3,5016,5016 0.09269532561302185
4,0.430426,000270c9100de789,14676,14676 0.43042585253715515
5,0.416876,0002b0fab5d3ccc4,14529,14529 0.41687577962875366
6,0.450706,000396be3c24830a,14436,14436 0.4507061541080475
7,0.095696,000506dc6ab3a40e,13104,13104 0.09569557756185532
8,0.237152,0005292fc4b005a3,14178,14178 0.23715221881866455
9,0.955915,0005456a82264bc8,14350,14350 0.9559152126312256


In [44]:
df

,id,landmarks
0,000088da12d664db,14591 0.2243746519088745
1,0001623c6d808702,14187 0.6876228451728821
2,0001bbb682d45002,14442 0.23606419563293457
3,0002362830cfe3a3,5016 0.09269532561302185
4,000270c9100de789,14676 0.43042585253715515
5,0002b0fab5d3ccc4,14529 0.41687577962875366
6,000396be3c24830a,14436 0.4507061541080475
7,000506dc6ab3a40e,13104 0.09569557756185532
8,0005292fc4b005a3,14178 0.23715221881866455
9,0005456a82264bc8,14350 0.9559152126312256


In [43]:
df_sample = pd.read_csv('sample_submission.csv')[['id']]
df_final = df_sample.merge(df, how = 'left', on = 'id')
df_final.to_csv('submission18.csv', index = False)

In [6]:
with open('../cl3403/pickle/X_train3.pickle', mode = 'rb') as f:
    X_train3 = pickle.load(f)
with open('../cl3403/pickle/X_train4.pickle', mode = 'rb') as f:
    X_train4 = pickle.load(f)
with open('../cl3403/pickle/X_train5.pickle', mode = 'rb') as f:
    X_train5 = pickle.load(f)

In [8]:
with open('../cl3403/pickle/X_truetest.pickle', mode = 'rb') as f:
    X_truetest = pickle.load(f)

In [9]:
X_train = np.vstack((X_train1, X_train2))
del X_train1, X_train2

In [10]:
X_train = np.vstack((X_train, X_train3))
del X_train3

In [11]:
X_train = np.vstack((X_train, X_train4))
del X_train4

In [12]:
X_train = np.vstack((X_train, X_train5))
del X_train5

In [13]:
y_train = np.hstack((y_train1, y_train2))
y_train = np.hstack((y_train, y_train3))
y_train = np.hstack((y_train, y_train4))
y_train = np.hstack((y_train, y_train5))

In [14]:
del y_train1,y_train2,y_train3,y_train4,y_train5

In [42]:
y_train.shape

(497500,)

In [64]:
X_train1 = np.append(X_train1, [X_train[0]])
X_train2 = np.append(X_train2, [X_train[1]])
y_train1 = np.append(y_train1, y_train[0])
y_train2 = np.append(y_train2, y_train[0])

In [65]:
X_train1.shape

(9007104,)

In [58]:
y_train1.shape

(2,)

In [66]:
X_train.shape

(497500, 64, 64, 3)

In [72]:
arr = []
arr2 = []

In [73]:
for i in range(len(X_train)):
    if (i % 10000 == 0):
        print(i)
    if (i % 2 == 0):
        arr.append(i)
    else:
        arr2.append(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000


In [78]:
X_train1 = X_train[arr]

In [84]:
X_train1.shape

(248750, 64, 64, 3)

In [81]:
y_train1 = y_train[arr]

In [83]:
y_train1.shape

(248750,)

In [85]:
X_train2 = X_train[arr2]

In [86]:
X_train2.shape

(248750, 64, 64, 3)

In [87]:
y_train2 = y_train[arr2]

In [88]:
y_train2.shape

(248750,)

In [90]:
import pickle
with open('X_train_true1.pickle', 'wb') as f:
    pickle.dump(X_train1, f, protocol=4)

KeyboardInterrupt: 

In [ ]:
with open('y_train_true1.pickle', 'wb') as f:
    pickle.dump(y_train1, f, protocol=4)

In [ ]:
with open('X_train_true2.pickle', 'wb') as f:
    pickle.dump(X_train2, f, protocol=4)

In [ ]:
with open('y_train_true2.pickle', 'wb') as f:
    pickle.dump(y_train2, f, protocol=4)

In [93]:
del X_train, y_train

In [100]:
X_train = np.vstack((X_train6, X_train7))
del X_train6, X_train7

In [101]:
X_train = np.vstack((X_train, X_train8))
del X_train8

In [102]:
X_train = np.vstack((X_train, X_train9))
del X_train9

In [103]:
X_train = np.vstack((X_train, X_train10))
del X_train10

In [104]:
with open('../cl3403/pickle/y_train6.pickle', mode = 'rb') as f:
    y_train6 = pickle.load(f)
with open('../cl3403/pickle/y_train7.pickle', mode = 'rb') as f:
    y_train7 = pickle.load(f)
with open('../cl3403/pickle/y_train8.pickle', mode = 'rb') as f:
    y_train8 = pickle.load(f)
with open('../cl3403/pickle/y_train9.pickle', mode = 'rb') as f:
    y_train9 = pickle.load(f)
with open('../cl3403/pickle/y_train10.pickle', mode = 'rb') as f:
    y_train10 = pickle.load(f)

In [117]:
y_train = np.hstack((y_train6, y_train7))
y_train = np.hstack((y_train, y_train8))
y_train = np.hstack((y_train, y_train9))
y_train = np.hstack((y_train, y_train10))

In [119]:
y_train.shape

(434269,)

In [105]:
arr3 = []
arr4 = []
for i in range(len(X_train)):
    if (i % 2 == 0):
        arr3.append(i)
    else:
        arr4.append(i)

In [114]:
X_train3 = X_train[arr3]

In [115]:
X_train4 = X_train[arr4]

In [120]:
y_train3 = y_train[arr3]

In [121]:
y_train4 = y_train[arr4]

In [122]:
del X_train, y_train

In [123]:
X_train = np.vstack((X_train1, X_train3))
del X_train1, X_train3

In [124]:
y_train = np.hstack((y_train1, y_train3))
del y_train1, y_train3

Epoch 1/20
465885/465885 [==============================] - 203s 436us/step - loss: 6.4867 - acc: 0.1885
Epoch 2/20
465885/465885 [==============================] - 190s 408us/step - loss: 4.0629 - acc: 0.4004
Epoch 3/20
465885/465885 [==============================] - 191s 409us/step - loss: 2.9017 - acc: 0.5495
Epoch 4/20
465885/465885 [==============================] - 190s 408us/step - loss: 2.2374 - acc: 0.6511
Epoch 5/20
465885/465885 [==============================] - 190s 408us/step - loss: 1.8103 - acc: 0.7282
Epoch 6/20
465885/465885 [==============================] - 191s 409us/step - loss: 1.5211 - acc: 0.7857
Epoch 7/20
465885/465885 [==============================] - 190s 407us/step - loss: 1.3183 - acc: 0.8269
Epoch 8/20
465885/465885 [==============================] - 190s 408us/step - loss: 1.1709 - acc: 0.8579
Epoch 9/20
465885/465885 [==============================] - 190s 409us/step - loss: 1.0508 - acc: 0.8831
Epoch 10/20
465885/465885 [============================

In [169]:
y_tmp = np.array([y_train2])
y_train_transpose = y_tmp.T

In [170]:
hist = model.fit(X_train2, y_train_transpose, batch_size = 2000, epochs = 20) 

Epoch 1/20
100000/248750 [===========>..................] - ETA: 59s - loss: 2.1539 - acc: 0.6953 

KeyboardInterrupt: 

In [167]:
y_tmp4 = np.array([y_train4])
y_train_transpose4 = y_tmp4.T

In [168]:
hist = model.fit(X_train4, y_train_transpose4, batch_size = 2000, epochs = 20) 

Epoch 1/20
 34000/217134 [===>..........................] - ETA: 1:12 - loss: 3.9240 - acc: 0.4639

KeyboardInterrupt: 

In [157]:
df

,id,landmarks
0,000088da12d664db,13475 0.49331745505332947
1,0001623c6d808702,4934 0.21698716282844543
2,0001bbb682d45002,5649 0.28459131717681885
3,0002362830cfe3a3,12172 0.6467244625091553
4,000270c9100de789,8160 0.14062567055225372
5,0002b0fab5d3ccc4,2246 0.2630113363265991
6,000396be3c24830a,4829 0.6505535244941711
7,000506dc6ab3a40e,10855 0.8238791227340698
8,0005292fc4b005a3,6221 0.31651076674461365
9,0005456a82264bc8,11668 0.9465128183364868


In [159]:
df_sample = pd.read_csv('sample_submission.csv')[['id']]
df_final = df_sample.merge(df, how = 'left', on = 'id')
df_final.to_csv('submission17.csv', index = False)

In [160]:
df_final

,id,landmarks
0,000088da12d664db,13475 0.49331745505332947
1,0001623c6d808702,4934 0.21698716282844543
2,0001bbb682d45002,5649 0.28459131717681885
3,0002362830cfe3a3,12172 0.6467244625091553
4,000270c9100de789,8160 0.14062567055225372
5,0002b0fab5d3ccc4,2246 0.2630113363265991
6,000396be3c24830a,4829 0.6505535244941711
7,000506dc6ab3a40e,10855 0.8238791227340698
8,0005292fc4b005a3,6221 0.31651076674461365
9,0005456a82264bc8,11668 0.9465128183364868


In [147]:
X_train2

array([[[[0.34117647, 0.31372549, 0.2       ],
         [0.38431373, 0.35686275, 0.24313725],
         [0.35294118, 0.32941176, 0.20392157],
         ...,
         [0.34509804, 0.3372549 , 0.18823529],
         [0.39215686, 0.38431373, 0.23529412],
         [0.36078431, 0.35294118, 0.20392157]],

        [[0.37647059, 0.34901961, 0.23529412],
         [0.2627451 , 0.23529412, 0.12156863],
         [0.25882353, 0.23529412, 0.10196078],
         ...,
         [0.35686275, 0.34509804, 0.20784314],
         [0.40784314, 0.4       , 0.25098039],
         [0.34901961, 0.34117647, 0.19215686]],

        [[0.30196078, 0.28235294, 0.16470588],
         [0.28627451, 0.26666667, 0.14117647],
         [0.32156863, 0.30588235, 0.16078431],
         ...,
         [0.37254902, 0.36078431, 0.22352941],
         [0.41568627, 0.40784314, 0.25882353],
         [0.34509804, 0.3372549 , 0.18823529]],

        ...,

        [[0.51764706, 0.39607843, 0.28627451],
         [0.51764706, 0.39607843, 0.28627451]

In [133]:
X_train1 = X_train[0 : 232926]
X_train3 = X_train[232926 : ]
y_train1 = y_train[0 : 232926]
y_train3 = y_train[232926 : ]

In [145]:
del X_train1, X_train3, X_train, y_train

In [146]:
X_train = np.vstack((X_train2, X_train4))
del X_train2, X_train4

MemoryError: 

In [ ]:
y_train = np.hstack((y_train2, y_train4))
del y_train2, y_train4

In [ ]:
y_tmp = np.array([y_train])
y_train_transpose = y_tmp.T

In [ ]:
model.fit(X_train, y_train_transpose, batch_size = 2000, epochs = 20)

In [ ]:
with open('pickle/y_truetest.pickle', mode = 'rb') as f:
    y_truetest = pickle.load(f)

In [18]:
y_train

array([   0,    0,    0, ..., 7661, 7661, 7661])

In [16]:
X_train.shape

(497500, 64, 64, 3)

In [17]:
y_train.shape

(497500,)

(1, 497500)

(497500, 1)

In [44]:
y_train_transpose

array([[   0],
       [   0],
       [   0],
       ...,
       [7661],
       [7661],
       [7661]])

Epoch 1/20
497500/497500 [==============================] - 187s 377us/step - loss: 5.2677 - acc: 0.2915
Epoch 2/20
497500/497500 [==============================] - 179s 360us/step - loss: 2.9788 - acc: 0.5449
Epoch 3/20
497500/497500 [==============================] - 182s 366us/step - loss: 2.1087 - acc: 0.6767
Epoch 4/20
497500/497500 [==============================] - 179s 359us/step - loss: 1.6610 - acc: 0.7544
Epoch 5/20
497500/497500 [==============================] - 178s 358us/step - loss: 1.3841 - acc: 0.8078
Epoch 6/20
497500/497500 [==============================] - 180s 363us/step - loss: 1.2013 - acc: 0.8440
Epoch 7/20
497500/497500 [==============================] - 179s 359us/step - loss: 1.0697 - acc: 0.8710
Epoch 8/20
497500/497500 [==============================] - 178s 358us/step - loss: 0.9709 - acc: 0.8915
Epoch 9/20
497500/497500 [==============================] - 181s 363us/step - loss: 0.9024 - acc: 0.9055
Epoch 10/20
497500/497500 [============================

KeyboardInterrupt: 

In [52]:
pred = model.predict(X_truetest)

In [53]:
landmark_id_pred = pred.argmax(axis = 1)
confidence = pred.max(axis = 1)

df = pd.DataFrame({'id': np.array(y_truetest),
                   'landmark_id': landmark_id_pred,
                   'confidence': confidence})

In [153]:
id1 = df['id']
landmark_id1 = df['landmark_id']
confidence1 = df['confidence']

In [154]:
df['landmarks'] = df.apply(lambda x: str(x['landmark_id']) + ' ' + str(x['confidence']), axis = 1)

In [156]:
df = df.drop(['confidence', 'landmark_id'], axis = 1)

In [56]:
del X_train, y_train

In [57]:
df.to_csv('pred1.csv', index = False)

In [58]:
with open('../cl3403/pickle/X_train6.pickle', mode = 'rb') as f:
    X_train6 = pickle.load(f)

In [59]:
with open('../cl3403/pickle/X_train7.pickle', mode = 'rb') as f:
    X_train7 = pickle.load(f)

In [60]:
with open('../cl3403/pickle/X_train8.pickle', mode = 'rb') as f:
    X_train8 = pickle.load(f)

In [61]:
with open('../cl3403/pickle/X_train9.pickle', mode = 'rb') as f:
    X_train9 = pickle.load(f)

In [62]:
with open('../cl3403/pickle/X_train10.pickle', mode = 'rb') as f:
    X_train10 = pickle.load(f)

In [63]:
X_train = np.vstack((X_train6, X_train7))
del X_train6, X_train7
X_train = np.vstack((X_train, X_train8))
del X_train8
X_train = np.vstack((X_train, X_train9))
del X_train9
X_train = np.vstack((X_train, X_train10))
del X_train10

In [64]:
with open('../cl3403/pickle/y_train6.pickle', mode = 'rb') as f:
    y_train6 = pickle.load(f)
with open('../cl3403/pickle/y_train7.pickle', mode = 'rb') as f:
    y_train7 = pickle.load(f)
with open('../cl3403/pickle/y_train8.pickle', mode = 'rb') as f:
    y_train8 = pickle.load(f)
with open('../cl3403/pickle/y_train9.pickle', mode = 'rb') as f:
    y_train9 = pickle.load(f)
with open('../cl3403/pickle/y_train10.pickle', mode = 'rb') as f:
    y_train10 = pickle.load(f)


In [65]:
y_train = np.hstack((y_train6, y_train7))
y_train = np.hstack((y_train, y_train8))
y_train = np.hstack((y_train, y_train9))
y_train = np.hstack((y_train, y_train10))

In [66]:
y_train

array([ 7661,  7661,  7661, ..., 14950, 14950, 14950])

In [67]:
y_train = y_train - 7661

In [68]:
y_train

array([   0,    0,    0, ..., 7289, 7289, 7289])

In [69]:
model = ResnetBuilder.build_resnet_18((3, 64, 64), 7290)

In [70]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [71]:
y_tmp = np.array([y_train])

In [72]:
y_train_transpose = y_tmp.T

In [73]:
y_train_transpose

array([[   0],
       [   0],
       [   0],
       ...,
       [7289],
       [7289],
       [7289]])

In [74]:
hist = model.fit(X_train, y_train_transpose, batch_size = 2000, epochs = 20) 

Epoch 1/20
434269/434269 [==============================] - 164s 378us/step - loss: 5.5586 - acc: 0.2587
Epoch 2/20
434269/434269 [==============================] - 162s 374us/step - loss: 3.2141 - acc: 0.5086
Epoch 3/20
434269/434269 [==============================] - 159s 366us/step - loss: 2.2770 - acc: 0.6480
Epoch 4/20
434269/434269 [==============================] - 160s 369us/step - loss: 1.7777 - acc: 0.7339
Epoch 5/20
434269/434269 [==============================] - 166s 382us/step - loss: 1.4715 - acc: 0.7925
Epoch 6/20
434269/434269 [==============================] - 159s 367us/step - loss: 1.2598 - acc: 0.8354
Epoch 7/20
434269/434269 [==============================] - 158s 363us/step - loss: 1.1071 - acc: 0.8672
Epoch 8/20
434269/434269 [==============================] - 158s 363us/step - loss: 0.9945 - acc: 0.8907
Epoch 9/20
434269/434269 [==============================] - 158s 364us/step - loss: 0.9068 - acc: 0.9092
Epoch 10/20
434269/434269 [============================

KeyboardInterrupt: 

In [75]:
pred = model.predict(X_truetest)

In [76]:
landmark_id_pred = pred.argmax(axis = 1)

In [77]:
landmark_id_pred += 7661

In [80]:
landmark_id_pred

array([12172,  9333, 10900, ..., 14010,  8087, 14591])

In [81]:
df

,confidence,id,landmark_id
0,0.357752,000088da12d664db,4987
1,0.289623,0001623c6d808702,5400
2,0.322219,0001bbb682d45002,6574
3,0.938433,0002362830cfe3a3,878
4,0.675434,000270c9100de789,4131
5,0.270579,0002b0fab5d3ccc4,5470
6,0.215907,000396be3c24830a,4066
7,0.211945,000506dc6ab3a40e,5328
8,0.323481,0005292fc4b005a3,985
9,0.311507,0005456a82264bc8,2743


In [82]:
confidence = pred.max(axis = 1)

df2 = pd.DataFrame({'id': np.array(y_truetest),
                   'landmark_id': landmark_id_pred,
                   'confidence': confidence})

In [107]:
id2 = df2['id']
landmark_id2 = df2['landmark_id']
confidence2 = df2['confidence']

In [112]:
confidence1

0         0.357752
1         0.289623
2         0.322219
3         0.938433
4         0.675434
5         0.270579
6         0.215907
7         0.211945
8         0.323481
9         0.311507
10        0.263137
11        0.231138
12        0.991775
13        0.299230
14        0.371792
15        0.997600
16        0.383504
17        0.143121
18        0.640988
19        0.780586
20        0.791368
21        0.237528
22        0.282363
23        0.981185
24        0.157527
25        0.232110
26        0.883780
27        0.427024
28        0.882400
29        0.302495
            ...   
115750    0.363487
115751    0.514364
115752    0.585215
115753    0.530961
115754    0.265513
115755    0.181498
115756    0.598322
115757    0.380774
115758    0.133920
115759    0.628842
115760    0.321540
115761    0.776416
115762    0.381465
115763    0.280536
115764    0.376948
115765    0.613657
115766    0.489914
115767    0.236547
115768    0.681762
115769    0.766686
115770    0.937437
115771    0.

In [113]:
landmark_true = []
confidence_true = []
for i in range(len(id1)):
    if confidence1[i] <= confidence2[i]:
        landmark_true.append(landmark_id2[i])
        confidence_true.append(confidence2[i])
    else:
        landmark_true.append(landmark_id1[i])
        confidence_true.append(confidence1[i])

In [106]:
landmark_id2

0         4987
1         5400
2         6574
3          878
4         4131
5         5470
6         4066
7         5328
8          985
9         2743
10        6772
11        6438
12        4793
13        3264
14        7030
15        3096
16        4776
17        4168
18        3496
19        1878
20         264
21        4352
22        6566
23        1394
24        5529
25        5702
26        4987
27        3106
28         218
29        5554
          ... 
115750    5278
115751     218
115752    3439
115753    2386
115754    4793
115755    3482
115756    3021
115757    1054
115758    7000
115759      95
115760    1054
115761    5554
115762    6238
115763    4625
115764    1581
115765    6051
115766    4546
115767     447
115768    6487
115769    5529
115770    7383
115771    2611
115772    4793
115773    2999
115774     264
115775    2183
115776    2228
115777     290
115778    2814
115779    3283
Name: landmark_id, Length: 115780, dtype: int64

In [114]:
d = {'id': id1, 'confidence': confidence_true,'landmark_id': landmark_true }

In [115]:
df = pd.DataFrame(data = d)
df['landmarks'] = df.apply(lambda x: str(x['landmark_id']) + ' ' + str(x['confidence']), axis = 1)

In [116]:
df

,confidence,id,landmark_id,landmarks
0,0.722695,000088da12d664db,12172,12172 0.722694993019104
1,0.528872,0001623c6d808702,9333,9333 0.5288721323013306
2,0.639971,0001bbb682d45002,10900,10900 0.6399708390235901
3,0.938433,0002362830cfe3a3,878,878 0.938433051109314
4,0.675434,000270c9100de789,4131,4131 0.6754342913627625
5,0.444340,0002b0fab5d3ccc4,10409,10409 0.4443398118019104
6,0.529559,000396be3c24830a,14623,14623 0.529559314250946
7,0.916411,000506dc6ab3a40e,11619,11619 0.9164109826087952
8,0.577902,0005292fc4b005a3,11589,11589 0.5779020190238953
9,0.458375,0005456a82264bc8,8263,8263 0.4583745300769806


In [117]:
df = df.drop(['confidence', 'landmark_id'], axis = 1)

In [118]:
df

,id,landmarks
0,000088da12d664db,12172 0.722694993019104
1,0001623c6d808702,9333 0.5288721323013306
2,0001bbb682d45002,10900 0.6399708390235901
3,0002362830cfe3a3,878 0.938433051109314
4,000270c9100de789,4131 0.6754342913627625
5,0002b0fab5d3ccc4,10409 0.4443398118019104
6,000396be3c24830a,14623 0.529559314250946
7,000506dc6ab3a40e,11619 0.9164109826087952
8,0005292fc4b005a3,11589 0.5779020190238953
9,0005456a82264bc8,8263 0.4583745300769806


In [119]:
df_sample = pd.read_csv('sample_submission.csv')[['id']]
df_final = df_sample.merge(df, how = 'left', on = 'id')
df_final.to_csv('submission16.csv', index = False)